This notebook goes through the coordinate reference systems (CRS) for both WRF and ERA5. 
In this way, we can plot both datasets side-by-side for comparison. 

In [5]:
# plotting stuff
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf
from wrf import (to_np, getvar, ALL_TIMES, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

# Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
else:
    sys.path.append('/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF

# Import the optwrf module
import optwrf
from optwrf import runwrf

In [6]:
# Verbose setting
verbose = True

# Define the datestr and paramstr
datestr = '2011-12-13'
paramstr = '19mp4lw4sw7lsm8pbl99cu'
wrffile_name = f'wrfout_processed_d01_{datestr}_{paramstr}.nc'
wrffile_name_orig = f'wrfout_d01_{datestr}_{paramstr}.nc'

# Find the absolute file path to your optwrf package
optwrf_abspath = os.path.dirname(os.path.abspath(inspect.getfile(optwrf)))
optwrf_abspath

# Open the original wrfout data (to use wrf-python I need the format from here)
wrffile_orig = os.path.join(optwrf_abspath, 'data', wrffile_name_orig)
wrfdata_orig = netCDF4.Dataset(wrffile_orig)
if verbose:
    print(f'Original WRF DATA:\n{wrfdata_orig}\n')

# Open the processed wrfout data
wrffile = os.path.join(optwrf_abspath, 'data', wrffile_name)
wrfdata = xr.open_dataset(wrffile)
if verbose:
    print(f'Processeed WRF DATA:\n{wrfdata}\n')
    
# Open the processed ERA5 data
erafile = os.path.join(optwrf_abspath, 'data', 'ERA5_EastUS_WPD-GHI_2011-12.nc')
eradata = xr.open_dataset(erafile)
if verbose:
    print(f'ERA5 DATA:\n{eradata}')

Original WRF DATA:
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE: 2011-12-13_00:00:00
    SIMULATION_START_DATE: 2011-12-13_00:00:00
    WEST-EAST_GRID_DIMENSION: 192
    SOUTH-NORTH_GRID_DIMENSION: 192
    BOTTOM-TOP_GRID_DIMENSION: 36
    DX: 12000.0
    DY: 12000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 0
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 19
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 1
    SF_SURFACE_PHYSICS: 7
    BL_PBL_PHYSICS: 8
    CU_PHYSICS: 99
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 

In [9]:
# It's easy to get the projection (also using get_cartopy()) if you have the original wrfout file,
# but since I didn't know about this at the time, It's difficult to get it from the processed wrfout file...
dni_orig = getvar(wrfdata_orig, "SWDDNI")
get_cartopy(dni_orig)

In [10]:
# Basically this wrf-python utility extracts the following attributes from the file
proj_parms = wrf.util.get_proj_params(wrfdata_orig)
proj_parms

{'MAP_PROJ': 1,
 'CEN_LAT': 40.259846,
 'CEN_LON': -81.64166,
 'TRUELAT1': 33.0,
 'TRUELAT2': 45.0,
 'MOAD_CEN_LAT': 40.259846,
 'STAND_LON': -97.0,
 'POLE_LAT': 90.0,
 'POLE_LON': 0.0,
 'DX': 12000.0,
 'DY': 12000.0}

In [11]:
# Which can be used to get the projection 
wrf_proj1 = wrf.projection.getproj(**proj_parms)
print(wrf_proj1)
print(f'Variable type: {type(wrf.projection.getproj(**proj_parms))}')

LambertConformal(stand_lon=-97.0, moad_cen_lat=40.25984573364258, truelat1=33.0, truelat2=45.0, pole_lat=90.0, pole_lon=0.0)
Variable type: <class 'wrf.projection.LambertConformal'>


In [12]:
# And then used to create the cartopy coordinate reference system.

# Set cutoff to -30 for NH, +30.0 for SH.
cutoff = -30.0 if wrf_proj1.moad_cen_lat >= 0 else 30.0

proj_cartopy = ccrs.LambertConformal(
    central_longitude=wrf_proj1.stand_lon,
    central_latitude=wrf_proj1.moad_cen_lat,
    standard_parallels=wrf_proj1._std_parallels,
    cutoff=cutoff)
proj_cartopy

In [13]:
# I'd really like to get it just from the processed wrfout file,
# buuuut I had to change the 'projection' attribute to a string to get the NetCDF file to write.
# Therefore, I have to do this tedious string parsing below to get the projection from the processed wrfout file.
try:
    wrf_proj_params = wrfdata.dni.attrs['projection']
except AttributeError:
    raise ValueError("Variable does not contain projection information")
wrf_proj_params = wrf_proj_params.replace('(', ', ')
wrf_proj_params = wrf_proj_params.replace(')', '')
wrf_proj_params = wrf_proj_params.split(',')
print(wrf_proj_params)
wrf_proj = wrf_proj_params[0]
stand_lon = float(wrf_proj_params[1].split('=')[1])
moad_cen_lat = float(wrf_proj_params[2].split('=')[1])
truelat1 = float(wrf_proj_params[3].split('=')[1])
truelat2 = float(wrf_proj_params[4].split('=')[1])
pole_lat = float(wrf_proj_params[5].split('=')[1])
pole_lon = float(wrf_proj_params[6].split('=')[1])

['LambertConformal', ' stand_lon=-97.0', ' moad_cen_lat=40.25984573364258', ' truelat1=33.0', ' truelat2=45.0', ' pole_lat=90.0', ' pole_lon=0.0']


In [14]:
# Fortunately, it still apppears to work.
if wrf_proj == 'LambertConformal':
    wrf_cartopy_proj = ccrs.LambertConformal(
                                            central_longitude=stand_lon,
                                            central_latitude=moad_cen_lat,
                                            standard_parallels=[truelat1, truelat2],
                                            )
else: 
    print('Check yourself...')
wrf_cartopy_proj

In [18]:
eradata

<xarray.Dataset>
Dimensions:    (Time: 744, latitude: 97, longitude: 129)
Coordinates:
  * Time       (Time) datetime64[ns] 2011-12-01 ... 2011-12-31T23:00:00
  * latitude   (latitude) float64 52.0 51.75 51.5 51.25 ... 28.5 28.25 28.0
  * longitude  (longitude) float64 264.0 264.2 264.5 264.8 ... 295.5 295.8 296.0
Data variables:
    VAR_100U   (Time, latitude, longitude) float32 ...
    VAR_100V   (Time, latitude, longitude) float32 ...
    WPD        (Time, latitude, longitude) float32 ...
    SSRD       (Time, latitude, longitude) float32 ...
    GHI        (Time, latitude, longitude) float32 ...

In [21]:
# The ERA5 data simply has this grid information
try:
    era_proj_params = eradata.VAR_100U.attrs['grid_specification']
    print(era_proj_params)
except AttributeError:
    raise ValueError("Variable does not contain grid_specification information")

0.25 degree x 0.25 degree from 90N to 90S and 0E to 359.75E (721 x 1440 Latitude/Longitude)


In [24]:
# Therefore, we can simply use a Plate Carree projection
era5_cartopy_proj = ccrs.PlateCarree()
era5_cartopy_proj